In [122]:
"""for pulling down data from hr.info API on response level and sending to CSV"""

import urllib.request, json
import csv
import itertools
import math
from collections import OrderedDict

import grequests
import pandas as pd
from pandas.io.json import json_normalize

In [23]:
#pulling data

def pull_indiv_rw(data):
    """
    return all individual crisis data, exclude ones that don't have a 200 status code.
    
    pull into a df and return, then that gets merged with the existing df
    """
    hrefz = data['rw_gen.href']

    def exception_handler(request, exception):
         print('Bad URL for ' + str(request))

    resps = []
    it = 200
    for v in range(0, len(hrefz), it):
        print('Pulling individual for hrefs to ' + str(v))
        rs = (grequests.get(ref) for ref in hrefz[v : v+it])
        resps += grequests.map(rs, exception_handler = exception_handler)
    
    resps = [json.loads(r.content) for r in resps if r.status_code == 200]
        
    return json_normalize(resps)[['data', 'href', 'totalCount']].add_prefix('rw_gen.')
#     return json_normalize(resps, ['totalCount', 'data', 'href']).add_prefix('rw_gen.')

def fetch_api_rw(maxv = None):
    """
    pull down all API info for RW general crisis and return as dataframe.
    
    maxv = (start_val, end_val)
    """
    data = []    
    
    if maxv:
        start = maxv[0]
        fin = maxv[1]
        step = min(fin - start, 1000)
    
    else:
        start = 0
        fin = json.loads(urllib.request.urlopen("https://api.reliefweb.int/v1/" \
                                   "disasters?appname=vocabulary&preset=external").read().decode())['totalCount']
        step = 1000
        
    for i in range(start, fin, step):
        with urllib.request.urlopen("https://api.reliefweb.int/v1/disasters?appname=vocabulary"
                                    "&preset=external&limit={0}&offset={1}"
                                    .format(min(step, fin - i), i)) as url:

            data += json.loads(url.read().decode())['data']


    ret = json_normalize(data).add_prefix('rw_gen.')
    
    assert(len(ret) == fin - start)
    print('pulling down from rw entry count of: ' + str(len(ret)))
    return ret
    

In [41]:
rs = (grequests.get(ref) for ref in ['https://api.reliefweb.int/v1/disasters/45654'])
resps = grequests.map(rs)

resps = [json.loads(r.content) for r in resps if r.status_code == 200]


j = json_normalize(resps[0]['data'])

,fields.country,fields.current,fields.date.created,fields.description,fields.description-html,fields.featured,fields.glide,fields.id,fields.name,fields.primary_country.href,...,fields.primary_type.code,fields.primary_type.id,fields.primary_type.name,fields.profile.overview,fields.profile.overview-html,fields.status,fields.type,fields.url,fields.url_alias,id
0,[{'href': 'https://api.reliefweb.int/v1/countr...,True,2018-04-18T00:00:00+00:00,Heavy rainfall occurred in different parts of ...,<p>Heavy rainfall occurred in different parts ...,False,FL-2018-000043-TZA,45654,Tanzania: Floods - Apr 2018,https://api.reliefweb.int/v1/countries/244,...,FL,4611,Flood,Heavy rainfall occurred in different parts of ...,<p>Heavy rainfall occurred in different parts ...,current,"[{'id': 4611, 'name': 'Flood', 'code': 'FL', '...",https://reliefweb.int/taxonomy/term/45654,https://reliefweb.int/disaster/fl-2018-000043-tza,45654


https://api.reliefweb.int/v1/disasters?appname=vocabulary&preset=external&limit=1&offset=10
pulling down from rw entry count of: 1
trim data length: 1
Pulling individual for hrefs to 0
Pulling individual for hrefs to 0


,rw_gen.data,rw_gen.href,rw_gen.totalCount
0,"[{'fields': {'id': 45654, 'name': 'Tanzania: F...",https://api.reliefweb.int/v1/disasters/45654,1


In [248]:
r

,rw_gen.fields.glide,rw_gen.fields.id,rw_gen.fields.name,rw_gen.fields.status,rw_gen.href,rw_gen.id,rw_gen.score,rw_gen.month,rw_gen.year,data.fields.description,data.fields.url_alias,data.primary_type.id,data.primary_type.name,data.primary_type.code,num_country,data.fields.country.name,data.fields.country.iso,data.fields.country.href,data.fields.country.location_lat,data.fields.country.location_long
0,TS-2007-000042-SLB,6291,Solomon Islands: Earthquake and Tsunami,past,https://api.reliefweb.int/v1/disasters/6291,6291,1,Apr,2007,None,None,None,None,None,1,Solomon Islands,slb,https://api.reliefweb.int/v1/countries/215,-8.9200,159.6300
1,FL-2007-000044-ARG,6190,Argentina: Floods,past,https://api.reliefweb.int/v1/disasters/6190,6190,1,Mar,2007,None,None,None,None,None,1,Argentina,arg,https://api.reliefweb.int/v1/countries/22,-35.3800,-64.0000
2,AC-2007-000041-PSE,5355,OPT: Waste Water Treatment Plant Floods,past,https://api.reliefweb.int/v1/disasters/5355,5355,1,Mar,2007,None,None,None,None,None,2,Israel,isr,https://api.reliefweb.int/v1/countries/125,31.0900,34.8700
3,FL-2007-000040-HTI,5150,Haiti: Floods,past,https://api.reliefweb.int/v1/disasters/5150,5150,1,Mar,2007,None,None,None,None,None,1,Haiti,hti,https://api.reliefweb.int/v1/countries/113,19.1800,-72.4300
4,LS-2007-000037-PAK,5758,South Asia: Floods and landslides,past,https://api.reliefweb.int/v1/disasters/5758,5758,1,Mar,2007,None,None,None,None,None,2,India,ind,https://api.reliefweb.int/v1/countries/119,22.7600,79.2800
5,FL-2007-000035-AFG,6646,Afghanistan: Floods and avalanche,past,https://api.reliefweb.int/v1/disasters/6646,6646,1,Mar,2007,None,None,None,None,None,1,Afghanistan,afg,https://api.reliefweb.int/v1/countries/13,33.8400,66.0300
6,TC-2007-000034-MDG,5936,Madagascar: Cyclone Indlala,past,https://api.reliefweb.int/v1/disasters/5936,5936,1,Mar,2007,None,None,None,None,None,1,Madagascar,mdg,https://api.reliefweb.int/v1/countries/144,-19.3700,46.7100
7,EQ-2007-000033-IDN,5211,Indonesia: Earthquake,past,https://api.reliefweb.int/v1/disasters/5211,5211,1,Mar,2007,None,None,None,None,None,1,Indonesia,idn,https://api.reliefweb.int/v1/countries/120,-2.2800,117.3700
8,LS-2007-000032-IDN,5201,Indonesia: Landslides,past,https://api.reliefweb.int/v1/disasters/5201,5201,1,Mar,2007,None,None,None,None,None,1,Indonesia,idn,https://api.reliefweb.int/v1/countries/120,-2.2800,117.3700
9,EP-2007-000031-PRY,6248,Paraguay: Dengue Outbreak,past,https://api.reliefweb.int/v1/disasters/6248,6248,1,Feb,2007,None,None,None,None,None,1,Paraguay,pry,https://api.reliefweb.int/v1/countries/186,-23.2400,-58.3900


In [82]:
nd = rw(test = None, year = 2005)

r = nd.master_pull()

r

https://api.reliefweb.int/v1/disasters?appname=vocabulary&preset=external&limit=1000&offset=0
https://api.reliefweb.int/v1/disasters?appname=vocabulary&preset=external&limit=1000&offset=1000
https://api.reliefweb.int/v1/disasters?appname=vocabulary&preset=external&limit=1000&offset=2000
https://api.reliefweb.int/v1/disasters?appname=vocabulary&preset=external&limit=21&offset=3000
pulling down from rw entry count of: 3021
trim data length: 1701
Pulling individual for hrefs to 0
Pulling individual for hrefs to 200
Pulling individual for hrefs to 400
Pulling individual for hrefs to 600
Pulling individual for hrefs to 800
Pulling individual for hrefs to 1000
Pulling individual for hrefs to 1200
Pulling individual for hrefs to 1400
Pulling individual for hrefs to 1600


ValueError: Columns must be same length as key

In [178]:
len(l)

30

In [181]:
l

,rw_gen.data,rw_gen.href,rw_gen.totalCount
0,"[{'fields': {'id': 6291, 'name': 'Solomon Isla...",https://api.reliefweb.int/v1/disasters/6291,1
1,"[{'fields': {'id': 6190, 'name': 'Argentina: F...",https://api.reliefweb.int/v1/disasters/6190,1
2,"[{'fields': {'id': 5355, 'name': 'OPT: Waste W...",https://api.reliefweb.int/v1/disasters/5355,1
3,"[{'fields': {'id': 5150, 'name': 'Haiti: Flood...",https://api.reliefweb.int/v1/disasters/5150,1
4,"[{'fields': {'id': 5758, 'name': 'South Asia: ...",https://api.reliefweb.int/v1/disasters/5758,1
5,"[{'fields': {'id': 6646, 'name': 'Afghanistan:...",https://api.reliefweb.int/v1/disasters/6646,1
6,"[{'fields': {'id': 5936, 'name': 'Madagascar: ...",https://api.reliefweb.int/v1/disasters/5936,1
7,"[{'fields': {'id': 5211, 'name': 'Indonesia: E...",https://api.reliefweb.int/v1/disasters/5211,1
8,"[{'fields': {'id': 5201, 'name': 'Indonesia: L...",https://api.reliefweb.int/v1/disasters/5201,1
9,"[{'fields': {'id': 6248, 'name': 'Paraguay: De...",https://api.reliefweb.int/v1/disasters/6248,1


In [184]:
nd = rw(test = (1450,1680), year = 2005)
r = nd.master_pull()

l = pull_indiv_rw(r)
j = json_normalize(l['rw_gen.data'][0])


for p in l:
    print(json_normalize(p['rw_gen.data'][0]))
#     print(json_normalize(l['rw_gen.data'][0])['fields.country'][0])
#     for v in json_normalize(l['rw_gen.data'][0]):
#         print(v)
#         if 'primary' in v:
#             primary = v
#             break

print(j['fields.country'][0])
        
# [v for v in j.iteritems()]

j

https://api.reliefweb.int/v1/disasters?appname=vocabulary&preset=external&limit=230&offset=1450
pulling down from rw entry count of: 230
trim data length: 230
Pulling individual for hrefs to 0
Pulling individual for hrefs to 200
Pulling individual for hrefs to 0
Pulling individual for hrefs to 200


TypeError: string indices must be integers

In [194]:
j = json_normalize(l['rw_gen.data'][0])
j

,fields.country,fields.current,fields.date.created,fields.featured,fields.glide,fields.id,fields.name,fields.primary_country.href,fields.primary_country.id,fields.primary_country.iso3,...,fields.primary_country.location.lon,fields.primary_country.name,fields.primary_type.code,fields.primary_type.id,fields.primary_type.name,fields.status,fields.type,fields.url,fields.url_alias,id
0,[{'href': 'https://api.reliefweb.int/v1/countr...,False,2007-04-02T00:00:00+00:00,False,TS-2007-000042-SLB,6291,Solomon Islands: Earthquake and Tsunami - Apr ...,https://api.reliefweb.int/v1/countries/215,215,slb,...,159.63,Solomon Islands,TS,4687,Tsunami,past,"[{'id': 4687, 'name': 'Tsunami', 'code': 'TS',...",https://reliefweb.int/taxonomy/term/6291,https://reliefweb.int/disaster/ts-2007-000042-slb,6291


In [251]:
nd = rw(test = False, year = 2005)
r = nd.master_pull()

https://api.reliefweb.int/v1/disasters?appname=vocabulary&preset=external&limit=1000&offset=0
https://api.reliefweb.int/v1/disasters?appname=vocabulary&preset=external&limit=1000&offset=1000
https://api.reliefweb.int/v1/disasters?appname=vocabulary&preset=external&limit=1000&offset=2000
https://api.reliefweb.int/v1/disasters?appname=vocabulary&preset=external&limit=21&offset=3000
pulling down from rw entry count of: 3021
trim data length: 1701
Pulling individual for hrefs to 0
Pulling individual for hrefs to 200
Pulling individual for hrefs to 400
Pulling individual for hrefs to 600
Pulling individual for hrefs to 800
Pulling individual for hrefs to 1000
Pulling individual for hrefs to 1200
Pulling individual for hrefs to 1400
Pulling individual for hrefs to 1600


ValueError: Columns must be same length as key

In [247]:
nd = rw(test = (1450,1480), year = 2005)
r = nd.master_pull()

https://api.reliefweb.int/v1/disasters?appname=vocabulary&preset=external&limit=30&offset=1450
pulling down from rw entry count of: 30
trim data length: 30
Pulling individual for hrefs to 0


In [246]:
def get_spec_crisis(ent):
    """
    merge relevant crisis level data
    """
    global ret
    j = json_normalize(ent['rw_gen.data'])
    primary = None

    for v in j['fields.country'][0]:
        if 'primary' in v:
            primary = v
            break

    #bad news if no primary
    if not primary:
        print('*** no primary! ' + str(v))
        ret = [None] * (len(self.all_cols) - len(self.new_cols_top))


    as_int = lambda x : print(j[x])
    as_is = lambda x : j[x][0] if x in j.columns else None
    len_country = lambda x : len(j['fields.country']) if 'fields.country' in j else None
    from_prim = lambda x : primary[x] if x in primary else None
    def from_prim_loc(col):
        ret = None
        if col == 'fields.country.location_lat':
            if 'location' in primary:
                return primary['location']['lat']
        elif col == 'fields.country.location_long':
            if 'location' in primary:
                return primary['location']['lon']



    p = lambda x : None

    get_vals = OrderedDict({
    #         'rw_gen.totalCount' : as_int,
            'fields.description' : as_is,
            'fields.url_alias' : as_is,
            'fields.primary_type.id' : as_is,
            'fields.primary_type.name' : as_is,
            'fields.primary_type.code' : as_is,
            'fields.country.name' : from_prim,
            'fields.country.iso' : from_prim,
            'fields.country.href' : from_prim,
            'fields.country.location_lat' : from_prim_loc,
            'fields.country.location_long' : from_prim_loc,
            'num_country' : len_country})

    ret = [{x, get_vals[x](x)} for x in get_vals.keys()] 
    
    
#     df = pd.DataFrame(columns = list(get_vals.keys()))
#     df.append(ret)



# get_spec_crisis()
#         l[list(get_vals.keys())] = l.apply(lambda x : pd.Series(self._get_spec_crisis_lamb(x)), axis = 1)

        #drop unnecessary columns, cleanup
#         l = l.drop('rw_gen.data', axis = 1)
#         l = l.drop('rw_gen.totalCount', axis = 1)
#         l['num_country'] = l['num_country'].astype(int)

#         self.data = self.data.merge(l, how = 'left', on = 'rw_gen.href')


https://api.reliefweb.int/v1/disasters?appname=vocabulary&preset=external&limit=30&offset=1450
pulling down from rw entry count of: 30
trim data length: 30
Pulling individual for hrefs to 0
Pulling individual for hrefs to 0


AttributeError: 'list' object has no attribute 'values'

In [244]:
ret = [{x, get_vals[x](x)} for x in get_vals.keys()]

[{'fields.description', None},
 {'fields.url_alias', 'https://reliefweb.int/disaster/eq-2006-000194-twn'},
 {4628, 'fields.primary_type.id'},
 {'Earthquake', 'fields.primary_type.name'},
 {'EQ', 'fields.primary_type.code'},
 {'fields.country.name', None},
 {'fields.country.iso', None},
 {None, 'fields.country.href'},
 {-8.92, 'fields.country.location_lat'},
 {'fields.country.location_long', 159.63},
 {'num_country', 1}]

In [234]:
[x for x in get_vals.keys()]

['fields.description',
 'fields.url_alias',
 'fields.primary_type.id',
 'fields.primary_type.name',
 'fields.primary_type.code',
 'fields.country.name',
 'fields.country.iso',
 'fields.country.href',
 'fields.country.location_lat',
 'fields.country.location_long',
 'num_country']

In [240]:
ret = [{x, get_vals[x](x)} for x in get_vals.keys()]

ret

[{'fields.description', None},
 {'fields.url_alias', 'https://reliefweb.int/disaster/eq-2006-000194-twn'},
 {4628, 'fields.primary_type.id'},
 {'Earthquake', 'fields.primary_type.name'},
 {'EQ', 'fields.primary_type.code'},
 {'fields.country.name', None},
 {'fields.country.iso', None},
 {None, 'fields.country.href'},
 {-8.92, 'fields.country.location_lat'},
 {'fields.country.location_long', 159.63},
 {'num_country', 1}]

In [231]:
rows_list = []
rows_list.append(ret)
pd.DataFrame(rows_list, columns = list(get_vals.keys()))

# df = pd.DataFrame(rows_list)               

# df
# df = df.append(pd.DataFrame(ret))
# df

,fields.description,fields.url_alias,fields.primary_type.id,fields.primary_type.name,fields.primary_type.code,fields.country.name,fields.country.iso,fields.country.href,fields.country.location_lat,fields.country.location_long,num_country
0,None,None,None,None,None,1,China - Taiwan Province,twn,https://api.reliefweb.int/v1/countries/61,23.6,121.02


In [202]:
ret

[None,
 None,
 None,
 None,
 None,
 1,
 'China - Taiwan Province',
 'twn',
 'https://api.reliefweb.int/v1/countries/61',
 23.6,
 121.02]

In [126]:
j

,fields.country,fields.current,fields.date.created,fields.description,fields.description-html,fields.featured,fields.glide,fields.id,fields.name,fields.primary_country.href,...,fields.primary_type.code,fields.primary_type.id,fields.primary_type.name,fields.profile.overview,fields.profile.overview-html,fields.status,fields.type,fields.url,fields.url_alias,id
0,[{'href': 'https://api.reliefweb.int/v1/countr...,True,2018-04-18T00:00:00+00:00,Heavy rainfall occurred in different parts of ...,<p>Heavy rainfall occurred in different parts ...,False,FL-2018-000043-TZA,45654,Tanzania: Floods - Apr 2018,https://api.reliefweb.int/v1/countries/244,...,FL,4611,Flood,Heavy rainfall occurred in different parts of ...,<p>Heavy rainfall occurred in different parts ...,current,"[{'id': 4611, 'name': 'Flood', 'code': 'FL', '...",https://reliefweb.int/taxonomy/term/45654,https://reliefweb.int/disaster/fl-2018-000043-tza,45654


In [193]:
l

,rw_gen.data,rw_gen.href,rw_gen.totalCount
0,"[{'fields': {'id': 6291, 'name': 'Solomon Isla...",https://api.reliefweb.int/v1/disasters/6291,1
1,"[{'fields': {'id': 6190, 'name': 'Argentina: F...",https://api.reliefweb.int/v1/disasters/6190,1
2,"[{'fields': {'id': 5355, 'name': 'OPT: Waste W...",https://api.reliefweb.int/v1/disasters/5355,1
3,"[{'fields': {'id': 5150, 'name': 'Haiti: Flood...",https://api.reliefweb.int/v1/disasters/5150,1
4,"[{'fields': {'id': 5758, 'name': 'South Asia: ...",https://api.reliefweb.int/v1/disasters/5758,1
5,"[{'fields': {'id': 6646, 'name': 'Afghanistan:...",https://api.reliefweb.int/v1/disasters/6646,1
6,"[{'fields': {'id': 5936, 'name': 'Madagascar: ...",https://api.reliefweb.int/v1/disasters/5936,1
7,"[{'fields': {'id': 5211, 'name': 'Indonesia: E...",https://api.reliefweb.int/v1/disasters/5211,1
8,"[{'fields': {'id': 5201, 'name': 'Indonesia: L...",https://api.reliefweb.int/v1/disasters/5201,1
9,"[{'fields': {'id': 6248, 'name': 'Paraguay: De...",https://api.reliefweb.int/v1/disasters/6248,1


In [ ]:
    def _get_spec_crisis_lamb(self, v):
        global ret
        global j

        ret = []

        if v['rw_gen.totalCount'] != 1:
            print('***wrong totalCount ' + str(v))

        j = json_normalize(v['rw_gen.data'][0])

        #add in top level data compents
        try:
            ret += [j[ent.split('data.')[1]][0] for ent in self.new_cols_top]
            
        except:
            ret += [None] * len(self.new_cols_top)

        #add in data.fields info. entry be like:
        """
            {'href': 'https://api.reliefweb.int/v1/countries/255',
             'id': 255,
             'iso3': 'yem',
             'location': {'lat': 15.94, 'lon': 47.62},
             'name': 'Yemen',
             'primary': True}
            primary = None
        """
        
        #add in other columns
        for v in j['fields.country'][0]:
            if 'primary' in v:
                primary = v
                break

        #bad news if no primary
        if not primary:
            print('*** no primary! ' + str(v))
            ret = [None] * (len(self.all_cols) - len(self.new_cols_top))
            
        else:
            try:
                rollback = ret
                
                #also add in other cols
                ret.append(len(j['fields.country'][0]))

                ret += [primary[c] for c in ['name', 'iso3', 'href']]
                ret.append(primary['location']['lat'])
                ret.append(primary['location']['lon'])
                
            except:
                ret = rollback + [None] * (len(rollback) - (len(self.all_cols) - len(self.new_cols_top)))
                
        return ret
    
    def get_spec_crisis(self):
        """
        merge relevant crisis level data
        """
        
        l = pull_indiv_rw(self.data)
        
        l['rw_gen.totalCount'] = l['rw_gen.totalCount'].astype(int)

        self.new_cols_top = 'data.fields.description',
                            'data.fields.url_alias',
                            'data.primary_type.id',
                            'data.primary_type.name',
                            'data.primary_type.code'
                            ]
    
        self.other_cols = ['num_country']
        
        self.new_cols_country = 'data.fields.country.name', 
                                'data.fields.country.iso', 
                                'data.fields.country.href',
                                'data.fields.country.location_lat', 
                                'data.fields.country.location_long']

        self.all_cols = self.new_cols_top + self.other_cols + self.new_cols_country

        for v in self.all_cols:
            l[v] = None
        
        l[self.all_cols] = l.apply(lambda x : pd.Series(self._get_spec_crisis_lamb(x)), axis = 1)

        #drop unnecessary columns, cleanup
        l = l.drop('rw_gen.data', axis = 1)
        l = l.drop('rw_gen.totalCount', axis = 1)
        l['num_country'] = l['num_country'].astype(int)

        self.data = self.data.merge(l, how = 'left', on = 'rw_gen.href')


In [81]:
class rw(object):
    
    def __init__(self, test, year):
        self.test = test
        self.data = None
        self.year = year

    def extract_date(self, val):
        name = val.replace(' ', '')[-7:]
        month = None
        year = None

        MIN_YEAR = 2005
        #if we don't have regular formatting, take just year
        if not name[0:3].isalpha():
            if not name[-4:].isnumeric():
                print("***bad year: " + name)
            else:
                year = int(name[-4:])
        else:
            month = name[:3]
            year = int(name[-4:])

        if year:
            if year >= MIN_YEAR:
                return [month, year]

        return (None, None)

    def get_date(self):
        self.data['rw_gen.month'] = None
        self.data['rw_gen.year'] = None
        self.data[['rw_gen.month', 'rw_gen.year']] = self.data.apply(lambda x: 
                                                            pd.Series(self.extract_date(x['rw_gen.fields.name'])), axis = 1)

        self.data = self.data[pd.notnull(self.data['rw_gen.year'])]
        self.data['rw_gen.year'] = self.data['rw_gen.year'].astype(int)
        
    def trim_nm(self):

        def trim(v):
            s_val = None

            if len(v.split('-')) != 1:
                s_val = '-'
            elif len(v.split('–')) != 1:
                s_val = '–' 

            if s_val:
                return v.split(s_val)[0]
            else:
                return v

        self.data['rw_gen.fields.name'] = self.data.apply(lambda x: trim(x['rw_gen.fields.name']), axis = 1)

    def rm_old(self):
        self.data = self.data.drop(self.data[self.data['rw_gen.year'] < self.year].index)
        print('trim data length: ' + str(len(self.data)))

    def get_spec_crisis(self):
        """
        merge relevant crisis level data
        """
        global ret
        j = json_normalize(ent['rw_gen.data'])
        primary = None

        for v in j['fields.country'][0]:
            if 'primary' in v:
                primary = v
                break

        #bad news if no primary
        if not primary:
            print('*** no primary! ' + str(v))
            ret = [None] * (len(self.all_cols) - len(self.new_cols_top))


        as_int = lambda x : print(j[x])
        as_is = lambda x : j[x][0] if x in j.columns else None
        len_country = lambda x : len(j['fields.country']) if 'fields.country' in j else None
        from_prim = lambda x : primary[x] if x in primary else None
        def from_prim_loc(col):
            ret = None
            if col == 'fields.country.location_lat':
                if 'location' in primary:
                    return primary['location']['lat']
            elif col == 'fields.country.location_long':
                if 'location' in primary:
                    return primary['location']['lon']



        p = lambda x : None

        get_vals = OrderedDict({
        #         'rw_gen.totalCount' : as_int,
                'fields.description' : as_is,
                'fields.url_alias' : as_is,
                'fields.primary_type.id' : as_is,
                'fields.primary_type.name' : as_is,
                'fields.primary_type.code' : as_is,
                'fields.country.name' : from_prim,
                'fields.country.iso' : from_prim,
                'fields.country.href' : from_prim,
                'fields.country.location_lat' : from_prim_loc,
                'fields.country.location_long' : from_prim_loc,
                'num_country' : len_country})

        ret = [{x, get_vals[x](x)} for x in get_vals.keys()] 


    #     df = pd.DataFrame(columns = list(get_vals.keys()))
    #     df.append(ret)



    # get_spec_crisis()
    #         l[list(get_vals.keys())] = l.apply(lambda x : pd.Series(self._get_spec_crisis_lamb(x)), axis = 1)

            #drop unnecessary columns, cleanup
    #         l = l.drop('rw_gen.data', axis = 1)
    #         l = l.drop('rw_gen.totalCount', axis = 1)
    #         l['num_country'] = l['num_country'].astype(int)

    #         self.data = self.data.merge(l, how = 'left', on = 'rw_gen.href')

        
    def master_pull(self):
        """take crises only after certain year, add month_crisis: mmm, and year_crisis: yyyy to each crisis's entry

            names are either in format of:
                MMM YYYY
                OR
                YYYY-YYYY

            if not in first format, check to see if end year > 2005
        """
        #only pull some data if test
        if self.test:
            self.data = fetch_api_rw(self.test)
        else:
            self.data = fetch_api_rw()

        #do things to primary rw data
        self.get_date()
        self.rm_old()
        self.trim_nm()
        
        #do crisiswise pull
        self.get_spec_crisis()
        
        return self.data